In [34]:
import Pkg

In [35]:
Pkg.status()

Status `~/.julia/environments/v1.10/Project.toml`
  [7073ff75] IJulia v1.24.2
  [2b0e0bc5] LanguageServer v4.5.1
  [0525e862] QuantumClifford v0.9.4 `../../../Documents/Repos/QuantumClifford.jl`
  [295af30f] Revise v3.5.14
  [37e2e46d] LinearAlgebra


In [5]:
using QuantumClifford

In [18]:
using QuantumClifford.ECC

## QECC interface 

In [60]:
# c = Shor9()
c = Toric(2,2)

Toric(2, 2)

In [61]:
parity_checks(c)

+ X_X_XX__
+ _X_XXX__
+ X_X___XX
+ ZZ__Z_Z_
+ ZZ___Z_Z
+ __ZZZ_Z_

Copy the functions that compute logical operators here. They are not exported in `QuantumClifford.jl/src/ecc/ECC.jl`.

In [62]:
"""Logical X operations of a code."""
function logx_ops(c)
    md = MixedDestabilizer(parity_checks(c))
    logicalxview(md)
end

"""Logical Z operations of a code."""
function logz_ops(c)
    md = MixedDestabilizer(parity_checks(c))
    logicalzview(md)
end

logz_ops

In [63]:
logx_ops(c)

+ __XX____
+ _____X_X

In [64]:
logz_ops(c)

+ _Z_Z____
+ ZZ__Z__Z

In [65]:
? logicalxview

search: logicalxview



A view of the subtableau corresponding to the logical X operators. See also [`tab`](@ref), [`stabilizerview`](@ref), [`destabilizerview`](@ref), [`logicalzview`](@ref)


Note that there is a redundency in logics. We may consider provide a API that accepct logical operators.

In [66]:
parity_checks_x(c)

3×8 SparseArrays.SparseMatrixCSC{Bool, Int64} with 12 stored entries:
 1  ⋅  1  ⋅  1  1  ⋅  ⋅
 ⋅  1  ⋅  1  1  1  ⋅  ⋅
 1  ⋅  1  ⋅  ⋅  ⋅  1  1

In [67]:
parity_checks_z(c)

3×8 SparseArrays.SparseMatrixCSC{Bool, Int64} with 12 stored entries:
 1  1  ⋅  ⋅  1  ⋅  1  ⋅
 1  1  ⋅  ⋅  ⋅  1  ⋅  1
 ⋅  ⋅  1  1  1  ⋅  1  ⋅

In [68]:
parity_checks(c)

+ X_X_XX__
+ _X_XXX__
+ X_X___XX
+ ZZ__Z_Z_
+ ZZ___Z_Z
+ __ZZZ_Z_

In [69]:
parity_checks(c)

+ X_X_XX__
+ _X_XXX__
+ X_X___XX
+ ZZ__Z_Z_
+ ZZ___Z_Z
+ __ZZZ_Z_

In [70]:
stab_to_gf2(parity_checks(c))

6×16 Matrix{Bool}:
 1  0  1  0  1  1  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  1  1  0  0  0  0  0  0  0  0  0  0
 1  0  1  0  0  0  1  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  1  0  0  1  0  1  0
 0  0  0  0  0  0  0  0  1  1  0  0  0  1  0  1
 0  0  0  0  0  0  0  0  0  0  1  1  1  0  1  0

## Concatenation

We consider cocantation of two small Toric codes.

In [108]:
c1 = Toric(2,2) # inner
c2 = Toric(2,2)# outer
k1 = code_k(c1)
k2 = code_k(c2)
n1 = code_n(c1)
n2 = code_n(c2)
s1 = code_s(c1)
s2 = code_s(c2)

6

In [168]:
n1, k1, s1

(8, 2, 6)

Each of the $n_2$ qubits of c2 is replaced by a $n_1$-qubit c1 code.

Two kinds of checks:

1. Parity checks of c1 codes. Their number is $n_2 (n_1-k_1)$.
2. Parity checks of the c2 code, where Pauli operators are replaced by c1 codes' logical operators. Their number is $(n_2-k_2) k_1$. Note that there are $k_2$ pairs of X and Z logics.

In total, there are $n_1 n_2 - k_1 k_2$ parity checks, consistent with the code parameter of the concatenated code.

The first type of checks:

In [109]:
parity_checks(c1)

+ X_X_XX__
+ _X_XXX__
+ X_X___XX
+ ZZ__Z_Z_
+ ZZ___Z_Z
+ __ZZZ_Z_

In [110]:
parity_checks(c1)[1]

+ X_X_XX__

In [111]:
I ⊗ parity_checks(c1)[1]  ⊗ I

+ _X_X_XX___

In [112]:
embed(n1*n2, 1:n1, parity_checks(c1)[1])

+ X_X_XX__________________________________________________________

In [113]:
vcat([embed(n1*n2, 1+(i-1)*n1:i*n1, parity_checks(c1)[j]) for i in 1:n2 for j in 1:s1])

48-element Vector{PauliOperator{Array{UInt8, 0}, Vector{UInt64}}}:
 + X_X_X⋯______
 + _X_XX⋯______
 + X_X__⋯______
 + ZZ__Z⋯______
 + ZZ___⋯______
 + __ZZZ⋯______
 + _____⋯______
 + _____⋯______
 + _____⋯______
 + _____⋯______
 + _____⋯______
 + _____⋯______
 + _____⋯______
 ⋮
 + _____⋯______
 + _____⋯______
 + _____⋯______
 + _____⋯______
 + _____⋯______
 + _____⋯______
 + _____⋯X_XX__
 + _____⋯_XXX__
 + _____⋯X___XX
 + _____⋯__Z_Z_
 + _____⋯___Z_Z
 + _____⋯ZZZ_Z_

The second types can be implemented by some kinds of tensor product on gf2 representation.

In [114]:
h2 = stab_to_gf2(parity_checks(c2))

6×16 Matrix{Bool}:
 1  0  1  0  1  1  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  1  1  0  0  0  0  0  0  0  0  0  0
 1  0  1  0  0  0  1  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  1  0  0  1  0  1  0
 0  0  0  0  0  0  0  0  1  1  0  0  0  1  0  1
 0  0  0  0  0  0  0  0  0  0  1  1  1  0  1  0

In [179]:
phases2 = phases(parity_checks(c2))

6-element Vector{UInt8}:
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00

Take the logics of c1.

In [138]:
vx1 = stab_to_gf2(logx_ops(c1))

2×16 Matrix{Bool}:
 0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  1  0  0  0  0  0  0  0  0

In [177]:
phases_x1 = phases(logx_ops(c1))

2-element view(::Vector{UInt8}, 7:8) with eltype UInt8:
 0x00
 0x00

In [137]:
vz1 = stab_to_gf2(logz_ops(c1))

2×16 Matrix{Bool}:
 0  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  1  1  0  0  1  0  0  1

In [178]:
phases_z1 = phases(logz_ops(c1))

2-element view(::Vector{UInt8}, 15:16) with eltype UInt8:
 0x00
 0x00

In [136]:
# logz_ops(c1)[1,1][1]

In [150]:
vcat(kron(h2[1,1:end÷2], vx1[1,1:end÷2]), kron(h2[1,end÷2+1:end], vz1[1,1:end÷2]))

128-element Vector{Bool}:
 0
 0
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [171]:
hlog = transpose(hcat([vcat(kron(h2[i,1:end÷2], vx1[j,1:end÷2]), kron(h2[i,end÷2+1:end], vz1[j,end÷2+1:end])) for i in 1:s2 for j in 1:k1]...))

12×128 transpose(::Matrix{Bool}) with eltype Bool:
 0  0  1  1  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  1  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  1  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  1  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  1  1  0  0  1  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  1  0  0  0  0  0  0  0  0

Then, we deal with the phase. If i=1, j=1:

In [225]:
UInt8((sum(h2[1,1:end÷2] * phases_x1[1]) + sum(h2[1,end÷2+1:end] * phases_z1[1]) + phases2[1]) & 0x3)

0x00

In [227]:
phases_log = [UInt8((sum(h2[1,1:end÷2] * phases_x1[1]) + sum(h2[1,end÷2+1:end] * phases_z1[1]) + phases2[1]) & 0x3) for i in 1:s2 for j in 1:k1]

12-element Vector{UInt8}:
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00
 0x00

In [232]:
Stabilizer(phases_log, hlog)

+ __XX______________XX______________XX______XX____________________
+ _____X_X_____________X_X_____________X_X_____X_X________________
+ __________XX______________XX______XX______XX____________________
+ _____________X_X_____________X_X_____X_X_____X_X________________
+ __XX______________XX______________________________XX______XX____
+ _____X_X_____________X_X_____________________________X_X_____X_X
+ _Z_Z_____Z_Z_____________________Z_Z_____________Z_Z____________
+ ZZ__Z__ZZZ__Z__Z________________ZZ__Z__Z________ZZ__Z__Z________
+ _Z_Z_____Z_Z_____________________________Z_Z_____________Z_Z____
+ ZZ__Z__ZZZ__Z__Z________________________ZZ__Z__Z________ZZ__Z__Z
+ _________________Z_Z_____Z_Z_____Z_Z_____________Z_Z____________
+ ________________ZZ__Z__ZZZ__Z__ZZZ__Z__Z________ZZ__Z__Z________

Next to do:

- Construct ECC data structure.
- What if the resulting code is not CSS? Some API design considerations.
- Design test.